In [1]:
import angr
import graphviz

In [2]:
path = './tests/2'
binary_path = f'{path}/out'

In [ ]:
p = angr.Project(binary_path, load_options={'auto_load_libs': True})
cfg = p.analyses.CFGFast()

In [4]:
def get_syscall_info(block_addr):
    block = p.factory.block(int(block_addr, base=16))
    inst_addrs = list(block.instruction_addrs)
    block_dump = block.bytes
    n = block.instructions
    prev = -2

    syscall_no = -1
    for i in range(n - 2, -1, -1):
        prev -= inst_addrs[i+1] - inst_addrs[i]
        if hex(block_dump[prev]) != '0xb8':
            continue
        lst = str(block_dump[prev + 1: prev + 5])[2:-1].split('\\x')
        # print(lst)
        syscall_no = int('0x' + lst[-1] + lst[-2] + lst[-3] + lst[-4], base=16)
        # print(syscall_no, int(syscall_no, base=16))
    return inst_addrs[-1], syscall_no

In [5]:
class func_info:
    def __init__(self, name, start, end, adj, callees, contains_syscall):
        self.name = name
        self.start = start
        self.end = end
        self.adj = adj
        self.callees = callees
        self.contains_syscall = contains_syscall
        
    
    def __str__(self):
        for u in self.adj:
            print(u, self.adj[u])
        s = f'''
            function name: {self.name}
            start = {self.start}
            end = {self.end}
            callees = {self.callees}
            #Nodes = {len(self.adj)}
            contains_syscall = {self.contains_syscall}
        '''
        return s
        

class tools:
    def __init__(self, cfg):
        self.cfg = cfg
        
#     def __init__(self, binary, load_libs = False):
#         p = angr.Project(binary, load_options={'auto_load_libs': load_libs}) 
#         self.cfg = p.analyses.CFGFast()


    def dfs(self, u, adj, visited, end):
        if visited[u]:
            return []
        visited[u] = True
        lst = []
        for x in adj[u]:
            if x[0] not in visited:
                print('unexpected')
            elif x[0] in end:
                lst.append(x)
            elif x[1] == 0:
                lst.extend(self.dfs(x[0], adj, visited, end))
            else:
                lst.append(x)
        return lst
    

    def compress(self, func):
        adj = func.adj
        queue = [func.start]
        adj2 = {}
        end = set(func.end)
        while len(queue) > 0:
            u = queue.pop(0)
            visited = {}
            for k in adj:
                visited[k] = False
            if u not in visited:
                print('unexpected 2')
                continue
            lst = self.dfs(u, adj, visited, end)
            for x in lst:
                if x[0] not in adj2:
                    queue.append(x[0])
            
            destinations = set()
            lst
            adj2[u] = lst
        
        fname = func.name
        start = func.start
        end = func.end
        callees = func.callees
        contains_syscall = func.contains_syscall
        return func_info(fname, start, end, adj2, callees, contains_syscall)
    

    def print_func_info(self, fname):
        fptr = self.cfg.functions.function(name = fname)
        
        G = fptr.transition_graph
        V = list(G.nodes())
        E = []
        for e in list(G.edges()):
            E.append(list(e))
        
        print()
        for v in V:
            print(v)
        print()
        for e in E:
            print(e)
        
        print(hex(fptr.addr))
        print(fptr.ret_sites)
        
    
    def get_func_info(self, fname):
        fptr = self.cfg.functions.function(name = fname)
        
        G = fptr.transition_graph
        V = list(G.nodes())
        E = []
        for e in list(G.edges()):
            E.append(list(e))
        
        adj = {}
        
        callees = set()
        
        pending = []
        
        for node in V:
            if isinstance(node, angr.codenode.BlockNode):
                u = node.addr
                u = hex(u)
                adj[u] = []
        
        for e in E:
            if not isinstance(e[1], angr.codenode.BlockNode):
                pending.append(e)
                continue
            
            u, v = e[0].addr, e[1].addr
            u, v = hex(u), hex(v)
            if (u not in adj) or (v not in adj):
                continue
            adj[u].append([v, 0])
        
        contains_syscall = False
        for e in pending:
            if not isinstance(e[1], angr.knowledge_plugins.functions.function.Function):
                continue
                
            u = hex(e[0].addr)
            if len(adj[u]) < 1:
                continue
            
            name = 'syscall'
            if not e[1].is_syscall:
                name = e[1].name
                if name == 'UnresolvableCallTarget':
                    if len(adj[u]) > 0:
                        adj[u].pop()
                    continue
                    
                callees.add(name)
                
                if adj[u][0][1] == 0:
                    adj[u][0][1] = 1
                    adj[u][0].append(name)
                else:
                    adj[u].append([adj[u][0][0], 1, name])
            else:
                contains_syscall = True
                addr, syscall_no = get_syscall_info(u)
                if adj[u][0][1] == 0:
                    adj[u][0][1] = 2
                    adj[u][0].extend([addr, syscall_no])
                else:
                    adj[u].append([adj[u][0][0], 2, addr, syscall_no])
        
        start = hex(fptr.addr)

        end = [hex(x.addr) for x in fptr.ret_sites]
        
        
        return func_info(fname, start, end, adj, callees, contains_syscall)
    
        
t = tools(cfg)          

In [6]:
def generateCFG():
    function_map = {}
    pending = ["main"]
    while len(pending) > 0:
        fname = pending.pop()
        if fname in function_map:
            continue
        finfo = t.get_func_info(fname)
#         print(finfo)
        pending.extend(list(finfo.callees))
        function_map[fname] = finfo
    return function_map    

In [7]:
function_map = generateCFG()
print(function_map)

{'main': <__main__.func_info object at 0x7fec854919c0>, '__libc_write': <__main__.func_info object at 0x7fec85491990>, '__pthread_disable_asynccancel': <__main__.func_info object at 0x7fec85491900>, '__pthread_enable_asynccancel': <__main__.func_info object at 0x7fec85491180>, 'close': <__main__.func_info object at 0x7fec854912a0>, '__open64': <__main__.func_info object at 0x7fec85491270>, 'factorial': <__main__.func_info object at 0x7fec85491570>, 'read': <__main__.func_info object at 0x7fec85491540>, 'fibonacci': <__main__.func_info object at 0x7fec85491240>}


In [8]:
compressed_map = {}
for name in function_map:
    compressed_map[name] = t.compress(function_map[name])
print(compressed_map)

{'main': <__main__.func_info object at 0x7fec85490730>, '__libc_write': <__main__.func_info object at 0x7fec85493940>, '__pthread_disable_asynccancel': <__main__.func_info object at 0x7fec85493a60>, '__pthread_enable_asynccancel': <__main__.func_info object at 0x7fec85493d90>, 'close': <__main__.func_info object at 0x7fec854938b0>, '__open64': <__main__.func_info object at 0x7fec85490760>, 'factorial': <__main__.func_info object at 0x7fec85490820>, 'read': <__main__.func_info object at 0x7fec85490880>, 'fibonacci': <__main__.func_info object at 0x7fec85493c40>}


In [9]:
def mergeCFG(function_map):
    adj = {}
    for fname in function_map:
        if len(function_map[fname].end) == 0:
            continue
        adjaceny = function_map[fname].adj
        for u in adjaceny:
            adj[u] = [x for x in adjaceny[u]]
    
    for u in adj:
        for i in range(len(adj[u])):
            
            # if unlabelled edge or syscall edge
            if adj[u][i][1] in [0, 2]:
                continue
            
            v = adj[u][i][0]
            name = adj[u][i][2]
            entry = function_map[name].start
            exits = function_map[name].end
            
            # if no exits in target function
            # then remove the edge
            if len(exits) == 0:
                adj[u][i] = [u, 0]
                # adj[u].pop(i)
                continue
            
            # update entry
            adj[u][i] = [entry, 0]
            
            # update exits
            for w in exits:
                # why doesnt work when if is removed
                if w in adj:
                    adj[w].append([v, 0])
    return adj
                    
                
# mergeCFG(function_map)            

In [10]:
def generateImage(adj, directory, filename):
    dot = graphviz.Digraph(name = filename)
    for u in adj:
        for x in adj[u]:
            v = x[0]
            if x[1] == 1:
                dot.edge(u, v, x[2])
            elif x[1] == 2:
                label = f'sycall:{x[3]},{hex(x[2])}'
                dot.edge(u, v, label)
            else:
                dot.edge(u, v)
    dot.render(directory = directory)

In [11]:
for name in function_map:
    generateImage(function_map[name].adj, f"{path}/cfg/function-wise", name)
    generateImage(compressed_map[name].adj, f"{path}/cfg/function-wise-compressed", name)

In [12]:
# Stats

adj = mergeCFG(function_map)
adj_c = mergeCFG(compressed_map)
e, e_c = 0, 0
v, v_c = 0, 0
v = len(adj)
v_c = len(adj_c)

for k in adj:
    e += len(adj[k])
for k in adj_c:
    e_c += len(adj_c[k])
print(f"Uncompressed: V = {v}, E = {e}")
print(f"Compressed-1: V = {v_c}, E = {e_c}")

Uncompressed: V = 89, E = 131
Compressed-1: V = 49, E = 85


In [13]:
generateImage(adj, f"{path}/cfg", "uncompressed")
generateImage(adj_c, f"{path}/cfg", "compressed-1")

In [14]:
compressed_cfg = func_info(
    "cfg-global-compressed",
    function_map["main"].start,
    function_map["main"].end, 
    adj,
    set(),
    False)

compressed_cfg = t.compress(compressed_cfg)
v_mc = len(compressed_cfg.adj)
e_mc = 0
for k in compressed_cfg.adj:
    e_mc += len(compressed_cfg.adj[k])
print(f"Compressed-2: V = {v_mc}, E = {e_mc}")

Compressed-2: V = 10, E = 23


In [15]:
generateImage(compressed_cfg.adj, f"{path}/cfg", "compressed-2")

In [16]:
with open(f"{path}/cfg/stats", 'w') as f:
    f.write(f"Uncompressed: V = {v}, E = {e}\n")
    f.write(f"Compressed-1: V = {v_c}, E = {e_c}\n")   
    f.write(f"Compressed-2: V = {v_mc}, E = {e_mc}\n")

In [17]:
def generateGraph(adj, start):
    blocks = adj.keys()
    mapping = {}
    
    for i, b in enumerate(blocks):
        mapping[b] = i
    
    new_adj = {}
    start = mapping[start]

    V = len(mapping)
    E = 0
    for u in adj:
        u1 = mapping[u]
        new_adj[u1] = []
        for x in adj[u]:
            v = x[0]
            v1 = mapping[v]
            sys_addr, sys_no = -1, -1
            if x[1] == 2:
                sys_addr, sys_no = x[2], x[3]
            new_adj[u1].append([v1, sys_addr, sys_no])
        E += len(new_adj[u1])

    
    
    print(new_adj)
    print(start)

    with open(f'{binary_path}.dot', 'w') as f:
        f.write(f'{V} {E} {start}\n')
        for u in new_adj:
            for x in new_adj[u]:
                f.write(f'{u} {x[0]} {x[1]} {x[2]}\n')
    
    

In [18]:
# Simple Merge
generateGraph(adj, function_map['main'].start)

{0: [[1, -1, -1], [2, -1, -1]], 1: [[49, -1, -1]], 2: [[49, -1, -1]], 3: [[5, -1, -1], [6, -1, -1]], 4: [[7, -1, -1], [8, -1, -1]], 5: [[9, -1, -1]], 6: [[81, -1, -1]], 7: [[9, -1, -1]], 8: [[70, -1, -1]], 9: [[12, -1, -1], [13, -1, -1]], 10: [[18, -1, -1]], 11: [[65, -1, -1]], 12: [], 13: [], 14: [[38, -1, -1]], 15: [[18, -1, -1]], 16: [[12, -1, -1], [13, -1, -1]], 17: [[14, -1, -1]], 18: [[19, -1, -1], [20, -1, -1]], 19: [[21, 4485877, 1]], 20: [[32, -1, -1]], 21: [[23, -1, -1], [24, -1, -1]], 22: [[25, 4485933, 1]], 23: [[14, -1, -1], [17, -1, -1]], 24: [[14, -1, -1], [17, -1, -1]], 25: [[26, -1, -1], [27, -1, -1]], 26: [[29, -1, -1]], 27: [[26, -1, -1]], 28: [[14, -1, -1], [17, -1, -1]], 29: [], 30: [[28, -1, -1], [48, -1, -1], [64, -1, -1], [80, -1, -1]], 31: [[28, -1, -1], [48, -1, -1], [64, -1, -1], [80, -1, -1]], 32: [[33, -1, -1], [34, -1, -1]], 33: [[35, -1, -1], [34, -1, -1]], 34: [[22, -1, -1], [42, -1, -1], [56, -1, -1], [74, -1, -1]], 35: [[36, -1, -1], [37, -1, -1]], 36:

In [19]:
# function wise compressed
generateGraph(adj_c, function_map['main'].start)

{0: [[29, -1, -1], [29, -1, -1]], 1: [[3, -1, -1], [45, -1, -1]], 2: [[3, -1, -1], [38, -1, -1]], 3: [], 4: [[10, -1, -1]], 5: [[35, -1, -1]], 6: [[22, -1, -1]], 7: [[10, -1, -1]], 8: [[3, -1, -1]], 9: [[22, -1, -1]], 10: [[11, 4485877, 1], [20, -1, -1]], 11: [[13, -1, -1], [14, -1, -1]], 12: [[15, 4485933, 1]], 13: [[6, -1, -1], [9, -1, -1]], 14: [[6, -1, -1], [9, -1, -1]], 15: [[17, -1, -1]], 16: [[6, -1, -1], [9, -1, -1]], 17: [[18, -1, -1], [19, -1, -1]], 18: [[16, -1, -1], [28, -1, -1], [34, -1, -1], [44, -1, -1]], 19: [[16, -1, -1], [28, -1, -1], [34, -1, -1], [44, -1, -1]], 20: [[21, -1, -1], [21, -1, -1]], 21: [[12, -1, -1], [24, -1, -1], [31, -1, -1], [40, -1, -1]], 22: [[23, 4486085, 3], [20, -1, -1]], 23: [[25, -1, -1], [26, -1, -1]], 24: [[27, 4486121, 3]], 25: [[8, -1, -1], [8, -1, -1]], 26: [[8, -1, -1], [8, -1, -1]], 27: [[17, -1, -1]], 28: [[8, -1, -1], [8, -1, -1]], 29: [[30, 4485481, 257], [20, -1, -1]], 30: [[32, -1, -1]], 31: [[33, 4485602, 257]], 32: [[1, -1, -1], 

In [20]:
# Global compressed
generateGraph(compressed_cfg.adj, function_map['main'].start)

{0: [[1, 4485481, 257], [2, 4485933, 1], [3, 4486121, 3], [4, 4485602, 257], [5, 4485770, -1]], 1: [[6, -1, -1], [7, 4485877, 1], [2, 4485933, 1], [3, 4486121, 3], [4, 4485602, 257], [5, 4485770, -1], [8, 4485712, -1]], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [[9, 4486085, 3], [2, 4485933, 1], [3, 4486121, 3], [4, 4485602, 257], [5, 4485770, -1]], 8: [[7, 4485877, 1], [2, 4485933, 1], [3, 4486121, 3], [4, 4485602, 257], [5, 4485770, -1]], 9: [[6, -1, -1]]}
0
